<a href="https://colab.research.google.com/github/manishkakarla/Research_accelerator/blob/master/Custom_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import tensorflow as tf
import numpy as np
from tensorflow.python.keras.utils.vis_utils import plot_model
import math
from tensorflow.keras.layers import Layer
from tensorflow import keras

In [92]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


In [111]:
from tensorflow.python.ops.gradients_impl import gradients
class IPC(keras.Model):

    def __init__(self, filters,k,alpha):
        '''Initializes the class and sets up the internal variables'''
        # YOUR CODE HERE
        super(IPC,self).__init__()
        #self.inputs=inputs
        #self.input_shap=self.inputs.shape
        self.filters=filters
        self.filters1=int(self.filters*alpha)
        self.filters2=int(self.filters*(1-alpha))
        self.k=k
        self.alpha=alpha
        self.z_i=None
        self.seed=None
        self.activated_beta=None
        self.input_shap=(8,8,512)
    
    def build(self,inputs):
        '''Create the state of the layer (weights)'''
        # a and b should be initialized with random normal, c (or the bias) with zeros.
        # remember to set these as trainable.
        # YOUR CODE HERE
        self.pointwise1=tf.keras.layers.Conv2D(self.filters2,1,strides=1,use_bias=False)
        
        seed_init=tf.random_uniform_initializer(minval=-1, maxval=0)
        seed_init_val=seed_init(shape=(1,1,self.input_shap[-1],1),dtype='float32')
        #seed_init_val=
        #initializer = RandomUniform(-1, 1)
        #config = initializer.get_config()
        #initializer = RandomUniform.from_config(config)
        self.seed=tf.Variable(initial_value=seed_init_val,trainable=True)
        self.seed=tf.placeholder_with_default(self.seed,[None]+[i for i in self.seed.shape[1:]])
        #self.z_i=tf.math.multiply(self.seed,self.alpha)
        self.z_i=self.seed*self.alpha
        

      #print(self.seed)
        #print(self.alpha)
        #self.Z_i=tf.math.multiply(self.seed,self.alpha)
        #print(self.alpha)
       # self.tanh=tf.math.tanh(self.beta)
        
        self.pointwise2=tf.keras.layers.Conv2D(self.filters1,1,strides=1,use_bias=False)
        self.convbeta=tf.keras.layers.Conv2D(self.k,(1,self.filters1+1),strides=1,padding='valid',use_bias=False)
        
      
   
    def call(self,inputs):
        '''Defines the computation from inputs to outputs'''
        # YOUR CODE HERE
        #x^2a + xb + c.
        inpu_shape=inputs.shape
        #print(self.z_i.shape)
        with tf.GradientTape() as tape:
          beta=self.convbeta(self.z_i)
        #print(f'beta shs{beta.shape}')
          self.activated_beta=tf.math.tanh(beta)
        #gradients=tape.gradient(se)
        pw1=self.pointwise1(inputs)
        print(f"pointwise1{pw1.shape}")
        shape1=pw1.shape
        pw1_reshape=tf.reshape(pw1,(1,shape1[1],shape1[2],shape1[3]))
        pw1=tf.placeholder_with_default(pw1,[None]+[i for i in pw1_reshape.shape[1:]])
        pw2=self.pointwise2(self.activated_beta)
       # 
        shape=pw2.shape
        pw2_reshape=tf.reshape(pw2,(1,inpu_shape[1],inpu_shape[2]
                                    ,(shape[1]*shape[2]*shape[3])//(inpu_shape[1]*inpu_shape[2])))
        
        pw= tf.placeholder_with_default(pw2_reshape, [None]+[i for i in pw2_reshape.shape[1:]])
        print(f"pointwiseshape2{pw.shape}")
       
        return tf.keras.layers.Concatenate(axis=3)([pw1, pw])

In [112]:
input=tf.keras.layers.Input((8,8,512) )
x1 = tf.keras.layers.DepthwiseConv2D(3, strides=1, padding='same',use_bias=False)(input)
print(x1)
a=IPC(filters=512,alpha=0.875,k=9)
#print(f'output{a(x1).shape}')
model= tf.keras.models.Model(input, outputs=a(x1))

Tensor("depthwise_conv2d_100/depthwise:0", shape=(?, 8, 8, 512), dtype=float32)
pointwise1(?, 8, 8, 64)
pointwiseshape2(?, 8, 8, 448)


In [113]:

model.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 8, 8, 512)]       0         
                                                                 
 depthwise_conv2d_100 (Depth  (None, 8, 8, 512)        4608      
 wiseConv2D)                                                     
                                                                 
 ipc_58 (IPC)                (None, 8, 8, 512)         40841     
                                                                 
Total params: 45,449
Trainable params: 45,449
Non-trainable params: 0
_________________________________________________________________


In [114]:
def mobilenet_block(x, f, s=1):
    x = tf.keras.layers.DepthwiseConv2D(3, strides=s, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(f, 1, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x =tf.keras.layers.Activation('relu')(x)
    return x

In [115]:

def ManishNet(input_shape, n_classes):
  """
  Manish's variation of MobileNetV1
  """
  input = tf.keras.layers.Input(input_shape)
  x = tf.keras.layers.Conv2D(32, 3, strides=1, padding='same')(input)
  #print("conv2d --1",x.shape)
  x = tf.keras.layers.BatchNormalization()(x)
  #print("batchNormalzation",x.shape)
  x = tf.keras.layers.Activation('relu')(x)
  #print("activation",x.shape)
  x = mobilenet_block(x, 64)
  x = mobilenet_block(x, 64,2)
  #print("1st mobilenetblock_1",x.shape)
  x = mobilenet_block(x, 128, 1)
  #print("2nd mobilenetblock_2",x.shape)
  x = mobilenet_block(x, 128)
  #print("3rd mobilenetblock_3",x.shape)

  x = mobilenet_block(x, 256)
  #print("4th mobilenetblock_4",x.shape)
  x = mobilenet_block(x, 256,2)
  #print("5th mobilenet_block_5",x.shape)

  x = mobilenet_block(x, 512, 1)
  
  #print("6th mobilenetblock_6",x.shape)
  for i in range(5):
    
    x = tf.keras.layers.DepthwiseConv2D(3, strides=1, padding='same',use_bias=False)(x)
    a = IPC(512,alpha=0.875,k=9)
    x=a(x)
   # print("jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj",x.shape)
    #print(f"{i+7} Improvedblock shape {x.shape}")
    
  
  x = tf.keras.layers.DepthwiseConv2D(3, strides=1, padding='same',use_bias=False)(x)
  a = IPC(512,alpha=0.875,k=9)(x)
 # x=a(x)
  #print("12 IPC",x.shape)

  x = tf.keras.layers.DepthwiseConv2D(3, strides=1, padding='same',use_bias=False)(x)
  a = IPC(512,alpha=0.875,k=9)(x)
  #x=a(x)
  print("13 IPC",x.shape)
  
  x = tf.keras.layers.GlobalAvgPool2D()(x)
  # x = tf.keras.layers.AveragePooling2D()(x)
  
  output = tf.keras.layers.Dense(n_classes, activation='softmax')(x)
  print(output.shape)
  
  model = tf.keras.models.Model(input, output)
  return model


In [116]:
input_shape=32,32,3
n_classes=10
model=ManishNet(input_shape, n_classes)

pointwise1(?, 8, 8, 64)
pointwiseshape2(?, 8, 8, 448)
pointwise1(?, 8, 8, 64)
pointwiseshape2(?, 8, 8, 448)
pointwise1(?, 8, 8, 64)
pointwiseshape2(?, 8, 8, 448)
pointwise1(?, 8, 8, 64)
pointwiseshape2(?, 8, 8, 448)
pointwise1(?, 8, 8, 64)
pointwiseshape2(?, 8, 8, 448)
pointwise1(?, 8, 8, 64)
pointwiseshape2(?, 8, 8, 448)
pointwise1(?, 8, 8, 64)
pointwiseshape2(?, 8, 8, 448)
13 IPC (?, 8, 8, 512)
(?, 10)


In [117]:
model.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_211 (Conv2D)         (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_90 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 activation_90 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 depthwise_conv2d_101 (Depth  (None, 32, 32, 32)       320       
 wiseConv2D)                                                     
                                                                 
 batch_normalization_91 (Bat  (None, 32, 32, 32)       128

https://stackoverflow.com/questions/54524124/dimension-mismatch-in-keras-during-model-fit

In [118]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


In [119]:
x_train=x_train.astype('float32')
x_train/=255
x_test=x_test.astype('float32')
x_test/=255
print(x_train.shape)

(50000, 32, 32, 3)


In [120]:
model.compile( loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam( learning_rate=0.001 ) , metrics=[ 'acc' ] )

In [121]:
from tensorflow.keras.utils import Sequence
import math
batch_size=32
epochs=1

class Generator(Sequence):
    # Class is a dataset wrapper for better training performance
    def __init__(self, x_set, y_set, batch_size=256):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.indices = np.arange(self.x.shape[0])

    def __len__(self):
        return math.floor(self.x.shape[0] / self.batch_size)

    def __getitem__(self, idx):
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = self.x[inds]
        batch_y = self.y[inds]
        return batch_x, batch_y

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

train_datagen = Generator(x_train, x_train, batch_size)
test_datagen = Generator(x_test, x_test, batch_size)

model.fit_generator(train_datagen,
    steps_per_epoch=len(x_train)//batch_size,
    validation_data=test_datagen,
    validation_steps=len(x_test)//batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: `model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


InvalidArgumentError: ignored

In [122]:
model.fit(x_train,y_train,shuffle=True,batch_size=32)

Train on 50000 samples


InvalidArgumentError: ignored

issue for the gradients after concatenation
https://github.com/tensorflow/tensorflow/issues/37726


* https://www.google.com/search?q=how+to+calculte+gradients+for+concatenate+in+tf&rlz=1C1GCEB_enUS937US937&oq=how+to+calculte+gradients+for+concatenate+in+tf&aqs=chrome..69i57j33i10i160l2.22259j0j7&sourceid=chrome&ie=UTF-8